# Library Import

In [ ]:
import pandas as pd
import httpx
from tqdm import tqdm

# Load Collection Table

In [ ]:
df_collection_dap_asis_dev = pd.read_csv(
    "dap-bc-db-asis-dev_collection_table_202308240910.csv"
)
del df_collection_dap_asis_dev["id"]
df_collection_dap_asis_dev

In [ ]:
df_collection_dap_asis_prod = pd.read_csv(
    "dap-bc-db-asis-prod_collection_table_202308240910.csv"
)
del df_collection_dap_asis_prod["id"]
df_collection_dap_asis_prod

In [ ]:
df_collection_dap_tobe_dev = pd.read_csv(
    "dap-bc-db-tobe-dev_collection_table_202308240923.csv"
)
del df_collection_dap_tobe_dev["id"]
df_collection_dap_tobe_dev["collection_id"] = df_collection_dap_tobe_dev[
    "collection_id"
].str.lower()
df_collection_dap_tobe_dev

In [ ]:
df_collection_town = pd.read_csv("nt_collection_master_202308221024.csv")
df_collection_town = df_collection_town.loc[
    df_collection_town["collection_id"].str.contains("0x")
]
del df_collection_town["idx"], df_collection_town["network"]

df_collection_town

# BnB Chain 추가

In [ ]:
df_collection_dap_asis_dev["network"].value_counts()

In [ ]:
df_collection_dap_asis_prod["network"].value_counts()

In [ ]:
df_collection_dap_tobe_dev["network"].value_counts()

> BNB Chain이 ASIS 및 TOBE에서 수집되어 있지 않기 때문에 추가할 수 없습니다.

# Network 추가

In [ ]:
df_collection_town = df_collection_town.merge(
    df_collection_dap_tobe_dev[["collection_id", "network"]],
    how="left",
    on="collection_id",
)
df_collection_town

In [ ]:
df_collection_town.loc[df_collection_town["network"].notnull()]

In [ ]:
df_collection_town["network"].value_counts()

In [ ]:
df_collection_town_notnetwork = df_collection_town.loc[
    df_collection_town["network"].isnull()
]
df_collection_town_notnetwork

In [ ]:
res = []
chain_domains = {"api": "ethereum", "api-polygon": "polygon"}
urls = [
    f"https://{chain_domain}.reservoir.tools/collections/v6"
    for chain_domain in chain_domains
]
headers = {
    "accept": "application/json",
    "x-api-key": "62ac45d0-e102-572c-bc88-d231a3e786b0",
}

for row in tqdm(df_collection_town_notnetwork.to_dict("records")):
    is_found = False
    for chain_domain, url in zip(chain_domains, urls):
        resp = httpx.get(
            url + f"id={row['collection_id']}",
            params={"collection_id": row["collection_id"]},
            headers=headers,
        ).json()
        print(resp)
        # if not resp["collections"] == []:
        #     if resp["collections"][0]["id"] in ["", None]:
        #         print(row["slug"])
        #     else:
        #         row["network"] = chain_domains[chain_domain]
        #         res.append(row)
        #         is_found = True
        #         break

    # if not is_found:
    #     res.append(row)

In [ ]:
df_collection_town_search = pd.DataFrame(res)
df_collection_town_search

In [ ]:
df_collection_town_search["network"].value_counts()

In [ ]:
df_collection_town.reset_index(inplace=True)
df_collection_town